In [1]:
import pyspark

In [2]:
sc = pyspark.SparkContext()
sc

22/12/14 19:24:35 WARN Utils: Your hostname, martin resolves to a loopback address: 127.0.1.1; using 10.27.42.174 instead (on interface wlp1s0)
22/12/14 19:24:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/14 19:24:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=pyspark-shell>